## Endpoint execution

This Jupyter Notebook simplifies interaction with the key endpoints of the Capture The Flag platform, offering a user-friendly environment to test and utilize its functionalities. It bridges the gap between backend mechanics and practical applications, making it a valuable resource for developers and enthusiasts.

### Initial imports and configurations

In [ ]:
import shutil
import requests
import urllib3

s = requests.session()

# Needed locally only
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
s.verify = False

# Mandatory configuration
keycloak_host = "https://ctf.jacopomauro.com/keycloak"
deployer_host = "https://deployer.ctf.jacopomauro.com"

# Paths to the challenge files
src_folder_location = "../challenge-ssh/src/"
solution_folder_location = "../challenge-ssh/solution/"
challenge_yaml_location = "../challenge-ssh/challenge.yml"

# Login credentials
username = ""
password = ""

### Get access token
Access token is valid for 5 minutes

In [ ]:
r = s.post(
    f"{keycloak_host}/realms/ctf/protocol/openid-connect/token/", 
    data={
        "client_id":"deployer", 
        "username": username, 
        "password": password, 
        "grant_type": "password", 
        "scope": "openid"
    }, 
    timeout=20
)
print("login:", r.status_code, r.content)
r.raise_for_status()

s.headers = {"Authorization": "Bearer " + r.json().get("access_token")}

### Post challenge to deployer service

In [ ]:
shutil.make_archive("challenge", "zip", src_folder_location)
shutil.make_archive("solution", "zip", solution_folder_location)

r = s.post(
    deployer_host + "/challenges", 
    files=[
        ("upload[]", open("challenge.zip", "rb")),
        ("upload[]", open(challenge_yaml_location, "rb")),
        ("upload[]", open("solution.zip", "rb"))
    ], 
    timeout=20
)
print("add challenge:", r.status_code, r.content)
r.raise_for_status()

challenge_id = r.json().get("challengeid")

### Submit challenge to CTFd

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/publish", timeout=20)
print("publish:", r.status_code, r.content)

### Start the Challenge using the API

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/start", timeout=20)
print("start challenge:", r.status_code, r.content)
r.raise_for_status()

### Start Test using the API

In [ ]:
r = s.post(deployer_host + f"/solutions/{challenge_id}/start", timeout=20)
print("start test:", r.status_code, r.content)
r.raise_for_status()

### Verify flag for a Challenge

In [ ]:
data = {
    "flag": "flag{ssh-example}"
}

r = s.post(deployer_host + f"/challenges/{challenge_id}/verify", timeout=20, json=data)
print("Test flag:", r.status_code, r.content)
r.raise_for_status()

### Stop the Challenge using the API

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/stop", timeout=20)
print("stop challenge:", r.status_code, r.content)
r.raise_for_status()

### Stop the Test using the API

In [ ]:
r = s.post(deployer_host + "/solutions/" + challenge_id + "/stop", timeout=20)
print("stop test:", r.status_code, r.content)
r.raise_for_status()

### Get Logs (not very beneficial)

In [ ]:
r = s.get(deployer_host + "/challenges/" + challenge_id + "/logs", timeout=20)
print("logs:", r.status_code, r.content)
r.raise_for_status()

### Check Challenge Status
```json
{"ready":"","secondsleft":"","started":"","url":"","verified":""}
```

In [ ]:
r = s.get(deployer_host + "/challenges/" + challenge_id + "/status", timeout=20)
print("logs:", r.status_code, r.content)
r.raise_for_status()

### SSH command

The following SSH command demonstrates how to connect to a CTF Challenge using SSH, leveraging the Bastion as a jump proxy while enabling port forwarding from the container to your local machine. This method allows access to a wider range of ports beyond those directly exposed by the platform itself.

```bash
ssh <username>@ssh.<subdomain> -p 8022 -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port>
sshpass -p 'password' ssh -o StrictHostKeyChecking=accept-new -p 8022 <user>@ssh.challenge-<subdomain> -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port> -N -f
```